In [3]:
import os
import sys
import re
import time
import random
import warnings
import collections
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

sys.path.append('../../src')
import cb_utils

sns.set(style="darkgrid")
pd.options.display.max_columns = 500

%load_ext autoreload
%autoreload 2

# Milliman PAC File ETL
1) log into SFTP site: https://secureftp.milliman.com  
2) In the "from milliman" folder, go to the latest directory  
     - In this case, the latest directory is "20221216"  
3) Make sure you have access to the following folder: \\Main Street Health\MSH_Strategy_Group - ACO Analyses\Raw MIlliman Flat FIles\  
     - create folder with same datestamp as file from milliman, in this case "20221216"  
4) Download the files from the SFTP site  
     - in this case there are 3  
5) copy the data disctionary to the folder on the network drive.  
6) download the prospective and retrospective CSV files into database tables in both the MSH and CB datagrip environments (Alan has a place he puts them)  
7) delete the last 30 columns from these database tables (we will create a "pivot file" with these 30 columns):  
8) with these 30 columns, run Alan's "pivot code" to create these files:  
     - strategic_milliman_prosp_20221128_pivot
     -  strategic_milliman_retro_20221128_pivot
9) copy 4 files to the "20221216" folder  
     - prospective flat file (without the 30 MA columns)
     - retrospective flat file (without the 30 MA columns)
     - prospective MA pivot file
     - retrospective MA pivor file
10) email Ali and Ben that process is complete with the following summaries that you can pull via queries and paste in excel into the same folder.  

### Config

In [4]:
data_dir = '/Users/bp/workspace/msh/milliman_data'
current_folder = '20240508'
pro_file_name = 'ACO Builder MSSP Prospective Explorer v2025.1.txt'
retro_file_name = 'ACO Builder MSSP Retrospective Explorer v2025.1.txt'

### Pro load

In [5]:
pro_df = pd.read_csv(f'{data_dir}/{current_folder}/{pro_file_name}',  encoding='latin1', low_memory=False, delimiter='\t')
pro_df = cb_utils.df_format_columns(pro_df)
print(f'{pro_df.shape[0]} rows, {pro_df.shape[1]} columns')
pro_df.head()

690422 rows, 179 columns


,pac_id,deid_tin,tin_public,year,program_type,provider_name,mssp_aco_name,reach_aco_name,practice_state,practice_msa,practice_st_msa,practice_msa_name,practice_zip,mailing_state,mailing_msa,mailing_st_msa,mailing_msa_name,mailing_zip,mailing_address,spec_count,np_count,cns_count,pcp_count,pa_count,other_count,unique_members,pys_esrd,pys_dis,pys_ad,pys_and,pys_total,rs_esrd_legacy,rs_dis_legacy,rs_ad_legacy,rs_and_legacy,risk_composite_legacy,rs_esrd_v24,rs_dis_v28,rs_ad_v28,rs_and_v28,risk_composite_v28,rs_esrd_demog,rs_dis_demog,rs_ad_demog,rs_and_demog,risk_composite_demog,paid_esrd,paid_dis,paid_ad,paid_and,paid_composite,reg_ret_exp_esrd,reg_ret_exp_dis,reg_ret_exp_ad,reg_ret_exp_and,reg_ret_exp_composite,regional_efficiency_ret,reg_pro_exp_esrd,reg_pro_exp_dis,reg_pro_exp_ad,reg_pro_exp_and,reg_pro_exp_composite,regional_efficiency_pro,reg_ret_exp_esrd_v24,reg_ret_exp_dis_v28,reg_ret_exp_ad_v28,reg_ret_exp_and_v28,reg_ret_exp_composite_v28,regional_efficiency_ret_v28,reg_pro_exp_esrd_v24,reg_pro_exp_dis_v28,reg_pro_exp_ad_v28,reg_pro_exp_and_v28,reg_pro_exp_composite_v28,regional_efficiency_pro_v28,prof_paid,op_paid,ip_paid,home_health_paid,snf_paid,hospice_paid,anc_paid,er_paid,sur_anes_paid,rad_path_paid,visits_paid,pharm_paid,dial_paid,other_prof_paid,pqem_cost,prof_paid_keep,op_paid_keep,sur_anes_paid_keep,rad_path_paid_keep,visits_paid_keep,pharm_paid_keep,dial_paid_keep,other_prof_paid_keep,pqem_cost_keep,op_visits,ip_admits,ip_days,home_health_visits,home_health_days,snf_admits,snf_days,hospice_visits,hospice_days,anc_services,er_visits,sur_anes_procs,rad_path_procs,pharm_procs,dial_visits,other_procs,op_visits_keep,sur_anes_procs_keep,rad_path_procs_keep,pharm_procs_keep,dial_visits_keep,other_procs_keep,awv,awv_keep,edcnnpa,edcnpa,epct,noner,injury,psych,alcohol,drug,unclassified,total_er,readmissions,unique_potl_members,pys_potl_esrd,pys_potl_dis,pys_potl_ad,pys_potl_and,paid_potl_esrd,paid_potl_dis,paid_potl_ad,paid_potl_and,rs_potl_esrd,rs_potl_dis,rs_potl_ad,rs_potl_and,pys_pcp_esrd,pys_pcp_dis,pys_pcp_ad,pys_pcp_and,paid_pcp_esrd,paid_pcp_dis,paid_pcp_ad,paid_pcp_and,rs_pcp_esrd,rs_pcp_dis,rs_pcp_ad,rs_pcp_and,pys_spc_esrd,pys_spc_dis,pys_spc_ad,pys_spc_and,paid_spc_esrd,paid_spc_dis,paid_spc_ad,paid_spc_and,rs_spc_esrd,rs_spc_dis,rs_spc_ad,rs_spc_and,pys_office,pys_outpatient,pys_snf,pys_nf,pys_hcbs,pys_alf,pys_fqhc_rhc,pys_other
0,9335446202,00004B7C4023,474454561,2017,MSSP Prospective,RINDFLEISCH FAMILY PRACTICE PLLC,NaN,"PEARL PRIMARY CARE NETWORK, LLC",Idaho,26820.0,ID-26820,"Idaho Falls, ID",83404,Idaho,26820.0,ID-26820,"Idaho Falls, ID",83404,"3155 CHANNING WAY STE A, IDAHO FALLS, ID 83404...",NaN,NaN,NaN,1.0,3.0,0.0,NaN,2.00,10.58,7.58,65.50,85.66,1.246027,1.054279,0.743298,1.192447,1.149580,1.148716,1.102218,0.781611,1.290791,1.225339,1.151278,1.067379,0.967948,1.119842,1.105532,53840.410000,18498.227788,14736.288918,19106.688855,19455.770021,77072.93,11693.71,17474.66,10279.96,12428.581291,1.361720,74214.50,11158.45,16987.21,9970.86,12022.555944,1.407708,77406.41,11550.35,17457.78,10028.83,12834.645036,1.318638,74788.00,11147.94,17101.71,9779.87,12490.726846,1.354945,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,191.0,2.0,33.416667,12.000000,127.333333,53840.412449,17571.782531,18292.490782,20448.687706,1.246027,1.360377,0.934429,1.395125,2.0,10.583333,7.583333,65.50,53840.412449,18492.401498,14729.810890,19106.688892,1.246027,1.053987,0.743116,1.192508,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.333333,0.0,0.0,0.0,3.0,21.333333,0.0,0.0
1,9335446202,00004B7C4023,474454561,2018,MSSP Prospective,RINDFLEISCH FAMILY PRACTICE PLLC,NaN,"PEARL PRIMARY CARE NETWORK, LLC",Idaho,26820.0,ID-26820,"Idaho Falls, ID",83404,Idaho,26820.0,ID-26820,"Idaho Falls, ID",83404,"3155 CHANNING WAY STE A, IDAHO FAL

In [6]:
conn = cb_utils.get_engine(source='msh_analytics')
pro_df.to_sql(f'milliman_pro_{current_folder}', conn, schema='raw', index=False, method='multi', chunksize=1000) # , if_exists='replace'

690422

In [7]:
ret_df = pd.read_csv(f'{data_dir}/{current_folder}/{retro_file_name}',  encoding='latin1', low_memory=False, delimiter='\t')
ret_df = cb_utils.df_format_columns(ret_df)
print(f'{ret_df.shape[0]} rows, {ret_df.shape[1]} columns')
ret_df.head()

669334 rows, 179 columns


,pac_id,deid_tin,tin_public,year,program_type,provider_name,mssp_aco_name,reach_aco_name,practice_state,practice_msa,practice_st_msa,practice_msa_name,practice_zip,mailing_state,mailing_msa,mailing_st_msa,mailing_msa_name,mailing_zip,mailing_address,spec_count,np_count,cns_count,pcp_count,pa_count,other_count,unique_members,pys_esrd,pys_dis,pys_ad,pys_and,pys_total,rs_esrd_legacy,rs_dis_legacy,rs_ad_legacy,rs_and_legacy,risk_composite_legacy,rs_esrd_v24,rs_dis_v28,rs_ad_v28,rs_and_v28,risk_composite_v28,rs_esrd_demog,rs_dis_demog,rs_ad_demog,rs_and_demog,risk_composite_demog,paid_esrd,paid_dis,paid_ad,paid_and,paid_composite,reg_ret_exp_esrd,reg_ret_exp_dis,reg_ret_exp_ad,reg_ret_exp_and,reg_ret_exp_composite,regional_efficiency_ret,reg_pro_exp_esrd,reg_pro_exp_dis,reg_pro_exp_ad,reg_pro_exp_and,reg_pro_exp_composite,regional_efficiency_pro,reg_ret_exp_esrd_v24,reg_ret_exp_dis_v28,reg_ret_exp_ad_v28,reg_ret_exp_and_v28,reg_ret_exp_composite_v28,regional_efficiency_ret_v28,reg_pro_exp_esrd_v24,reg_pro_exp_dis_v28,reg_pro_exp_ad_v28,reg_pro_exp_and_v28,reg_pro_exp_composite_v28,regional_efficiency_pro_v28,prof_paid,op_paid,ip_paid,home_health_paid,snf_paid,hospice_paid,anc_paid,er_paid,sur_anes_paid,rad_path_paid,visits_paid,pharm_paid,dial_paid,other_prof_paid,pqem_cost,prof_paid_keep,op_paid_keep,sur_anes_paid_keep,rad_path_paid_keep,visits_paid_keep,pharm_paid_keep,dial_paid_keep,other_prof_paid_keep,pqem_cost_keep,op_visits,ip_admits,ip_days,home_health_visits,home_health_days,snf_admits,snf_days,hospice_visits,hospice_days,anc_services,er_visits,sur_anes_procs,rad_path_procs,pharm_procs,dial_visits,other_procs,op_visits_keep,sur_anes_procs_keep,rad_path_procs_keep,pharm_procs_keep,dial_visits_keep,other_procs_keep,awv,awv_keep,edcnnpa,edcnpa,epct,noner,injury,psych,alcohol,drug,unclassified,total_er,readmissions,unique_potl_members,pys_potl_esrd,pys_potl_dis,pys_potl_ad,pys_potl_and,paid_potl_esrd,paid_potl_dis,paid_potl_ad,paid_potl_and,rs_potl_esrd,rs_potl_dis,rs_potl_ad,rs_potl_and,pys_pcp_esrd,pys_pcp_dis,pys_pcp_ad,pys_pcp_and,paid_pcp_esrd,paid_pcp_dis,paid_pcp_ad,paid_pcp_and,rs_pcp_esrd,rs_pcp_dis,rs_pcp_ad,rs_pcp_and,pys_spc_esrd,pys_spc_dis,pys_spc_ad,pys_spc_and,paid_spc_esrd,paid_spc_dis,paid_spc_ad,paid_spc_and,rs_spc_esrd,rs_spc_dis,rs_spc_ad,rs_spc_and,pys_office,pys_outpatient,pys_snf,pys_nf,pys_hcbs,pys_alf,pys_fqhc_rhc,pys_other
0,9335446202,00004B7C4023,474454561,2016,MSSP Retrospective,RINDFLEISCH FAMILY PRACTICE PLLC,NaN,"PEARL PRIMARY CARE NETWORK, LLC",Idaho,26820.0,ID-26820,"Idaho Falls, ID",83404,Idaho,26820.0,ID-26820,"Idaho Falls, ID",83404,"3155 CHANNING WAY STE A, IDAHO FALLS, ID 83404...",NaN,NaN,NaN,1.0,3.0,0.0,NaN,2.0,17.17,14.42,109.42,143.01,1.090726,0.971724,0.849401,1.073552,1.039837,1.090726,0.971724,0.849401,1.073552,1.039837,1.185739,0.978476,1.020923,1.150350,1.120466,56585.55,17498.055329,22268.219834,20561.795741,20869.814209,69928.69,10868.23,16501.34,10281.32,11725.860871,1.711625,0,0,0,0,0,0,69928.69,10868.23,16501.34,10281.32,11725.860871,1.711625,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,262.0,2.0,38.000000,16.750000,188.833333,56585.551531,17314.022256,26333.558964,23801.718526,1.090726,1.025304,0.935534,1.233379,2.0,17.166667,14.416667,109.416667,56585.551531,17501.453129,22273.368371,20562.422187,1.090726,0.971947,0.849591,1.073577,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.000000,0.0,0.0,0.0,2.333333,54.666667,0.0,0.0
1,9335446202,00004B7C4023,474454561,2017,MSSP Retrospective,RINDFLEISCH FAMILY PRACTICE PLLC,NaN,"PEARL PRIMARY CARE NETWORK, LLC",Idaho,26820.0,ID-26820,"Idaho Falls, ID",83404,Idaho,26820.0,ID-26820,"Idaho Falls, ID",83404,"3155 CHANNING WAY STE A, IDAHO FALLS, ID 83404...",NaN,NaN,NaN,1.0,3.0,0.0,NaN,1.0,20.83,14.50,150.42,186.75,0.

In [8]:
ret_df.to_sql(f'milliman_ret_{current_folder}', conn, schema='raw', index=False, method='multi', chunksize=1000) # if_exists='replace', 

669334

In [9]:
ret_df.year.value_counts()

2016    93392
2017    90403
2018    87384
2019    84748
2020    82510
2021    79560
2022    76027
2023    75310
Name: year, dtype: int64

In [10]:
for yr in [2016, 2017, 2018, 2019, 2020, 2021, 2022]:
    ret_df.loc[ret_df.year == yr].to_csv(f'{data_dir}/{current_folder}/ret_{yr}.csv',index=False)

# Did not use below

In [6]:
conn = cb_utils.get_engine()
pro_df.to_sql(f'milliman_pro_{current_folder}', conn, schema='strategic', index=False, method='multi', chunksize=1000) # , if_exists='replace'

613556

### Retro load

In [33]:

ret_df.to_csv('ret.csv',index=False)

In [10]:
conn = cb_utils.get_engine()
ret_df.to_sql(f'milliman_ret_{current_folder}', conn, schema='strategic', index=False, method='multi', chunksize=1000) # if_exists='replace', 

594838

### Pivots

In [11]:
def pivot_ma_plan_enrollment(df):
    dfs = []
    for i in range(1, 16):
        name, enrollment = f'ma_plan_{i}_name', f'ma_plan_{i}_enrollment'
        plan = df[['deid_tin', 'pac_id', 'year', name, enrollment]]
        plan = plan.loc[~(plan[name].isna()) & ~(plan[enrollment].isna())]
        plan.columns = ['deid_tin', 'pac_id', 'year', 'ma_plan_name', 'ma_enrollment']
        dfs.append(plan)
    return pd.concat(dfs)

In [12]:
ret_pivot_df = pivot_ma_plan_enrollment(ret_df)

KeyError: "['ma_plan_1_name', 'ma_plan_1_enrollment'] not in index"

In [ ]:
ret_pivot_df.head()

In [13]:
pro_pivot_df = pivot_ma_plan_enrollment(pro_df)

KeyError: "['ma_plan_1_name', 'ma_plan_1_enrollment'] not in index"

In [ ]:
pro_pivot_df.head()

### Summaries 
### Prospective

In [ ]:
s1 = ret_df[['year', 'unique_members']].groupby('year', as_index=False).sum()
s1.head()

### Retro

In [ ]:
s2 = pro_df[['year', 'unique_members']].groupby('year', as_index=False).sum()
s2.head()

### Prospective

In [ ]:
s3 = pro_pivot_df[['year', 'ma_plan_name', 'ma_enrollment']].groupby(['year', 'ma_plan_name'], as_index=False).sum().sort_values('ma_enrollment', ascending=False)
s3.head()

### Retro

In [ ]:
s4 = ret_pivot_df[['year', 'ma_plan_name', 'ma_enrollment']].groupby(['year', 'ma_plan_name'], as_index=False).sum().sort_values('ma_enrollment', ascending=False)
s4.head()

### Outputs

In [ ]:
ret_df.loc[:, ret_df.columns[:-30]].to_csv(f'{data_dir}/{current_folder}/ours_milliman_ret_{current_folder}.csv', index=False)

In [ ]:
pro_df.loc[:, pro_df.columns[:-30]].to_csv(f'{data_dir}/{current_folder}/ours_milliman_pro_{current_folder}.csv', index=False)

In [ ]:
pro_pivot_df.to_csv(f'{data_dir}/{current_folder}/pro_pivot.csv', index=False)
ret_pivot_df.to_csv(f'{data_dir}/{current_folder}/ret_pivot.csv', index=False)

In [ ]:
with pd.ExcelWriter(f'{data_dir}/{current_folder}/summary.xlsx') as writer:
    s1.to_excel(writer, sheet_name='retrospective', index=False)
    s2.to_excel(writer, sheet_name='prospective', index=False)
    s3.to_excel(writer, sheet_name='prospective_pivot', index=False)
    s4.to_excel(writer, sheet_name='retrospective_pivot', index=False)